In [0]:
# imports
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import StringType
import time

from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
# set up storage
blob_container = "container1" # The name of your container created in https://portal.azure.com
storage_account = "w261sec6group3" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261sec6group3_scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261sec6group3_key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
df_final = spark.read.parquet(f"{blob_url}/final_feature_eng_2022")

In [0]:
df_final = df_final.fillna(5, subset='pagerank')

In [0]:
df_final.write.mode('overwrite').parquet(f"{blob_url}/final_feature_eng_2022")

In [0]:
df_final = spark.read.parquet(f"{blob_url}/final_feature_eng")

In [0]:
CATEGORICAL_COLS = ['quarter', 'day_of_week', 'op_unique_carrier', 'tail_num', 'origin', 'origin_state_abr', 'dest', 'dest_state_abr', 'crs_dep_hour', 'flight_time_utc_hour', 'flight_time_utc_month', 'avg_hourly_wind_direction', \
                   'avg_hourly_wind_direction_lag6', 'avg_hourly_wind_direction_lag12', 'arr_del15_lag1', 'origin_lag1', 'is_holiday', 'frequent_delay', 'type']

NUMERICAL_COLS = ['crs_elapsed_time', 'pagerank', 'total_flights', \
                  'avg_hourly_dew_point_temp', 'avg_hourly_dry_bulb_temp', 'avg_hourly_relative_humidity', 'avg_hourly_station_pressure', 'avg_hourly_visibility', 'avg_hourly_wind_speed', 'avg_hourly_precipitation_ordinal', \
                  'avg_hourly_dew_point_temp_lag6', 'avg_hourly_dry_bulb_temp_lag6', 'avg_hourly_relative_humidity_lag6', 'avg_hourly_station_pressure_lag6', 'avg_hourly_visibility_lag6', 'avg_hourly_wind_speed_lag6', 'avg_hourly_precipitation_ordinal_lag6', \
                  'avg_hourly_dew_point_temp_lag12', 'avg_hourly_dry_bulb_temp_lag12', 'avg_hourly_relative_humidity_lag12', 'avg_hourly_station_pressure_lag12', 'avg_hourly_visibility_lag12', 'avg_hourly_wind_speed_lag12', 'avg_hourly_precipitation_ordinal_lag12']

In [0]:
# remove duplicates
df_final = df_final.dropDuplicates()
# drop null values in the dataframe
df_final = df_final.dropna()

In [0]:
def dataPreprocessing(df, categorical_cols, numerical_cols):
  """
  Preprocess the data.
  """
  # initiate list to feed into pipeline
  stages = []

  # instantiate string indexing and one hot encoding for each categorical feature
  for categoricalCol in categorical_cols:
      # instantiate string indexer
      stringIndexer = StringIndexer(inputCol=categoricalCol, 
                                    outputCol = categoricalCol + 'Index')
      # instantiate one hot encoder
      encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], 
                              outputCols=[categoricalCol + "classVec"])
      # add each instantiated string indexer and one hot encoder to stages list
      stages += [stringIndexer, encoder]

  # input columns for vector assembler
  assemblerInputs = [column + "classVec" for column in categorical_cols] + numerical_cols + ['flight_time_utc_year']
  # convert all columns into vectors
  assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
  # add vector assembler to stages list
  stages += [assembler]
  
  # instantiate pipeline to execute all actions in stages list
  pipeline = Pipeline(stages=stages)
  # fit all stages of the pipeline with dataframe
  pipelineModel = pipeline.fit(df)
  # get the vectorized features to put into classifier
  df = pipelineModel.transform(df)
  
  return df

In [0]:
df_final.display()

In [0]:
# preprocess the dataset for modelling
df_final = dataPreprocessing(df=df_final, categorical_cols=CATEGORICAL_COLS, numerical_cols=NUMERICAL_COLS)

In [0]:
df_final.display()

quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_fips,origin_state_nm,origin_wac,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_fips,dest_state_nm,dest_wac,crs_dep_time,dep_time,dep_delay,dep_delay_new,dep_del15,dep_delay_group,dep_time_blk,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,arr_del15,arr_delay_group,arr_time_blk,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,distance_group,year,IATA,station_id,name,icao,lat,lon,altitude,station_lon,station_lat,timezone,type,crs_dep_hour,flight_hour,flight_time,flight_time_utc,flight_time_utc_hour,flight_time_utc_year,flight_time_utc_month,flight_time_utc_date,is_holiday,weather_station,weather_station_name,metar_hour,avg_hourly_dew_point_temp,avg_hourly_dry_bulb_temp,avg_hourly_relative_humidity,avg_hourly_station_pressure,avg_hourly_visibility,avg_hourly_wind_direction,avg_hourly_wind_speed,avg_hourly_precipitation_ordinal,pagerank,avg_hourly_dew_point_temp_lag6,avg_hourly_dry_bulb_temp_lag6,avg_hourly_relative_humidity_lag6,avg_hourly_station_pressure_lag6,avg_hourly_visibility_lag6,avg_hourly_wind_direction_lag6,avg_hourly_wind_speed_lag6,avg_hourly_precipitation_ordinal_lag6,avg_hourly_dew_point_temp_lag12,avg_hourly_dry_bulb_temp_lag12,avg_hourly_relative_humidity_lag12,avg_hourly_station_pressure_lag12,avg_hourly_visibility_lag12,avg_hourly_wind_direction_lag12,avg_hourly_wind_speed_lag12,avg_hourly_precipitation_ordinal_lag12,arr_del15_lag1,origin_lag1,frequent_delay,total_flights
1,3,10,4,2022-03-10,AS,19930,AS,N618AS,51,10299,1029906,30299,ANC,"Anchorage, AK",AK,2,Alaska,1,10754,1075405,30107,BRW,"Barrow, AK",AK,2,Alaska,1,1438,1429,-9.0,0.0,0.0,-1,1400-1459,22.0,1451,1638,4.0,1634,1642,8.0,8.0,0.0,0,1600-1659,0.0,0.0,116.0,133.0,107.0,1.0,725.0,3,2022,ANC,70272526491,Ted Stevens Anchorage International Airport,PANC,61.174400329589844,-149.99600219726562,152,-149.966,61.178,America/Anchorage,large_airport,14.0,1438,2022-03-10T14:38:00.000+0000,2022-03-10T23:38:00.000+0000,23,2022,3,10,0,70272526491,"ANCHORAGE LAKE HOOD SEA PLANE BASE, AK US",2022-03-10T20:00:00.000+0000,31.0,32.0,96.0,29.389999389648438,10.0,30.0,5.0,0.0,3.6731414415331654,32.0,33.0,96.0,29.43666712443034,10.0,36.666666666666664,4.666666666666667,0.0,30.0,32.0,92.0,29.559999465942383,10.0,350.0,7.0,0.0,0.0,ANC,0,465782
1,3,13,7,2022-03-13,AS,19930,AS,N526AS,51,10299,1029906,30299,ANC,"Anchorage, AK",AK,2,Alaska,1,10754,1075405,30107,BRW,"Barrow, AK",AK,2,Alaska,1,1438,1453,15.0,15.0,1.0,1,1400-1459,12.0,1505,1646,3.0,1634,1649,15.0,15.0,1.0,1,1600-1659,0.0,0.0,116.0,116.0,101.0,1.0,725.0,3,2022,ANC,70272526491,Ted Stevens Anchorage International Airport,PANC,61.174400329589844,-149.99600219726562,152,-149.966,61.178,America/Anchorage,large_airport,14.0,1438,2022-03-13T14:38:00.000+0000,2022-03-13T22:38:00.000+0000,22,2022,3,13,0,70272526491,"ANCHORAGE LAKE HOOD SEA PLANE BASE, AK US",2022-03-13T19:00:00.000+0000,9.0,25.0,50.0,29.350000381469727,10.0,50.0,11.0,0.0,3.6731414415331654,13.0,29.0,51.0,29.350000381469727,10.0,30.0,18.0,0.0,13.0,26.0,57.0,29.40999984741211,10.0,20.0,16.0,0.0,0.0,ANC,0,465782
1,3,8,2,2022-03-08,DL,19790,DL,N342NW,1258,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14574,1457405,34574,ROA,"Roanoke, VA",VA,51,Virginia,38,2239,2306,27.0,27.0,1.0,1,2200-2259,23.0,2329,18,6.0,2356,24,28.0,28.0,1.0,1,2300-2359,0.0,0.0,77.0,78.0,49.0,1.0,357.0,2,2022,ATL,72219013874,Hartsfield Jackson Atlanta International Airport,KATL,33.6367,-84.428101,1026,-84.442,33.63,America/New_York,large_airport,22.0,2239,2022-03-08T22:39:00.000+0000,2022-03-09T03:39:00.000+0000,3,2022,3,9,0,72219013874,"ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",2022-03-09T00:00:00.000+0000,51

In [0]:
# store preprocessed dataset in blob storage
df_final.write.mode('overwrite').parquet(f"{blob_url}/final_vectorized_2022")